In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
DATA_FOLDER = '/beelife/data/'
DATA_FILE_KVG = 'euph_000014_vetcases_table_kvg.csv'
DATA_FILE_AT = 'euph_000014_vetcases_table_vetcaseat.csv'
DATA_FILE_BE = 'euph_000014_vetcases_table_vetcasebe.csv'
DATA_FILE_VETCASE = 'euph_000014_vetcases_table_vetcase.csv'

In [3]:
vetcases_df = pd.read_csv(os.path.join(DATA_FOLDER, DATA_FILE_VETCASE))

In [4]:
vetcases_df.head()

,id,year,month,casenumber,disease,animalspecies,datestart,dateend,country,nuts,districtcode,postcode,unit,value,annotation,sourceid
0,1,2015,1.0,TKH-2014-000443,American foulbrood of honey bees,1341976,2015-01-07,2015-05-04,AUT,AT12,307.0,NaN,number,1,NaN,50
1,2,2015,1.0,TKH-2014-000454,American foulbrood of honey bees,1341976,2015-01-12,NaN,AUT,AT21,205.0,NaN,number,1,NaN,61
2,3,2015,2.0,TKH-2015-000173,American foulbrood of honey bees,1341976,2015-02-16,2015-04-08,AUT,AT12,325.0,NaN,number,1,NaN,50
3,4,2015,3.0,TKH-2015-000335,American foulbrood of honey bees,1341976,2015-03-06,2015-04-09,AUT,AT12,309.0,NaN,number,1,NaN,50
4,5,2015,2.0,TKH-2015-000553,American foulbrood of honey bees,1341976,2015-02-12,NaN,AUT,AT21,205.0,NaN,number,1,NaN,61


In [5]:
vetcases_df.describe()

,id,year,month,animalspecies,districtcode,postcode,value,annotation,sourceid
count,597.000000,597.000000,585.000000,597.0,460.000000,72.000000,597.000000,0.0,597.000000
mean,299.000000,2017.388610,6.362393,1341976.0,489.580435,6327.625000,1.221106,NaN,51.247906
std,172.483333,2.257665,1.908512,0.0,168.783511,1720.032504,1.746912,NaN,19.748048
min,1.000000,2006.000000,1.000000,1341976.0,106.000000,1549.000000,0.000000,NaN,1.000000
25%,150.000000,2016.000000,5.000000,1341976.0,323.000000,6180.000000,1.000000,NaN,38.000000
50%,299.000000,2017.000000,6.000000,1341976.0,504.000000,6740.000000,1.000000,NaN,59.000000
75%,448.000000,2019.000000,8.000000,1341976.0,620.000000,7332.000000,1.000000,NaN,61.000000
max,597.000000,2022.000000,12.000000,1341976.0,900.000000,9255.000000,36.000000,NaN,80.000000


In [6]:
vetcases_df['disease'].value_counts()

disease
American foulbrood of honey bees                                                                                560
European foulbrood of honey bees                                                                                 35
Varroosis of honey bees in the case of imminent or actual death of at least 30% of the colonies in an apiary      2
Name: count, dtype: int64

In [7]:
print(vetcases_df.isnull().sum())

id                 0
year               0
month             12
casenumber       137
disease            0
animalspecies      0
datestart         65
dateend          172
country            0
nuts               0
districtcode     137
postcode         525
unit               0
value              0
annotation       597
sourceid           0
dtype: int64


In [10]:
import geopandas as gpd 
import folium 
# Load shapefile or GeoJSON with regional data (shapefile or geojson containing NUTS boundaries) 
# Provide the correct path to the shapefile 
gdf = gpd.read_file(shapefile) 
# Example: Filter by a specific NUTS code nuts_code = 'DE21' 
# Replace with the NUTS code you want 
region = gdf[gdf['NUTS_ID'] == "AT12"] 
# Get the centroid of the region for map centering 
centroid = region.geometry.centroid.iloc[0] 
latitude = centroid.y 
longitude = centroid.x 
# Create a folium map centered around the region 
m = folium.Map(location=[latitude, longitude], zoom_start=6) 
# Add the region to the map with a highlight (change the style as needed) 
folium.GeoJson( region, style_function=lambda x: { 'fillColor': 'orange', # Highlight color 
                                                  'color': 'black', # Border color 
                                                  'weight': 2, # Border width 
                                                  'fillOpacity': 0.5 # Fill opacity 
                                                 } ).add_to(m) 
# Display the map 
m.save("highlighted_region_map.html") # Save the map to an HTML file

In [52]:
import folium
import geopandas as gpd
from shapely.geometry import shape
from branca.colormap import linear

colormap = linear.RdBu_06.scale(0, 300)

print(colormap(5.0))

colormap

# To be beautified using: https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html


# Initialize the map (e.g., centered in Europe)
m = folium.Map(location=[50, 10], zoom_start=4)

# Load the shapefile
shapefile_path = "/beelife/data/NUTS_RG_20M_2024_3035.shp" 

gdf = gpd.read_file(shapefile_path)

# Filter the GeoDataFrame by NUTS code
nuts_code = 'AT12'  # Replace with the NUTS code you want to highlight

region_ids = vetcases_df['nuts'].unique()
np.random.shuffle(region_ids)

for i, nuts_code in enumerate(region_ids):
    region = gdf[gdf['NUTS_ID'] == nuts_code].copy()
    if len(region.index) == 0:
        continue
    region["colormapval"] = i*10
    # Add the region to the map (Convert the geometry into GeoJSON and add it as a feature)
    folium.GeoJson( region, style_function=lambda x: { 'fillColor': colormap(x["properties"]["colormapval"]), # Highlight color 
                                                  'color': 'black', # Border color 
                                                  'weight': 2, # Border width 
                                                  'fillOpacity': 0.5 # Fill opacity 
                                                 } ).add_to(m) 


'''
# Get the centroid of the region for map centering 
centroid = region.geometry.centroid.iloc[0] 
latitude = centroid.y 
longitude = centroid.x 
'''

# Display the map in the notebook
m


#b7212fff


In [20]:
# Save the map 
m.save("highlighted_region_map.html") # Save the map to an HTML file